In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import seaborn as sns
import re
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
import nltk
import torch
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/My Drive/')

In [5]:
!cp "Intent Classification"/* /content/

In [13]:
train_data = pd.read_csv('/content/preprocessed_train_data.csv')
test_data = pd.read_csv('/content/preprocessed_test_data.csv')

In [14]:
train_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explan edit made usernam hardcor metallica fan...,0,0,0,0,0,0
1,000103f0d9cfb60f,daww match background colour im seem stuck tha...,0,0,0,0,0,0
2,000113f07ec002fd,hey man im realli tri edit war guy constant re...,0,0,0,0,0,0
3,0001b41b1c6bb37e,cant make real suggest improv wonder section s...,0,0,0,0,0,0
4,0001d958c54c6e35,sir hero chanc rememb page that,0,0,0,0,0,0


In [6]:
X = train_data['comment_text'].values
y= train_data.drop(['id','comment_text'],axis=1).values

In [7]:
#!pip install sentence_transformers

In [8]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
model = model.to('cuda')

In [9]:
len(X)

159571

In [10]:
batch_size = 1024  # Adjust this value based on your available memory
all_embeddings = []

for i in range(0, len(X), batch_size):
    batch = X[i:i+batch_size]
    batch_embeddings = model.encode(batch, convert_to_tensor=True)
    all_embeddings.append(batch_embeddings)

X = torch.cat(all_embeddings, dim=0)

In [12]:
torch.save(X, '/content/lm_embeddings.pt')

In [9]:
X = torch.load('/content/lm_embeddings.pt', map_location=torch.device('cpu'))

In [16]:
X = X.numpy()

In [18]:
X.shape

(159571, 384)

In [19]:
y.shape

(159571, 6)

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.1,random_state= 12)

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [24]:
from sklearn.multioutput import MultiOutputClassifier  # For multi-label classification
from sklearn.multioutput import MultiOutputRegressor  # For multi-label regression
from sklearn.naive_bayes import MultinomialNB

logreg_clf = LogisticRegression(random_state=42,class_weight = 'balanced')

clf = MultiOutputClassifier(logreg_clf)  # Replace YourClassifier with the desired classifier
clf.fit(X_train, y_train)

MultiOutputClassifier(estimator=LogisticRegression(class_weight='balanced',
                                                   random_state=42))

In [25]:
# Predicting Train and Test data
log_prediction = clf.predict(X_test)
log_pred = clf.predict(X_train)
print(" Test Accuracy {0:.2f}%".format(100*accuracy_score(log_prediction,y_test)))
print(" Train Accuracy {0:.2f}%".format(100*accuracy_score(log_pred, y_train)))
print(classification_report(log_prediction,y_test))

 Test Accuracy 79.15%
 Train Accuracy 79.71%
              precision    recall  f1-score   support

           0       0.87      0.48      0.62      2922
           1       0.93      0.19      0.31       867
           2       0.92      0.43      0.59      1883
           3       0.84      0.04      0.08       787
           4       0.91      0.36      0.51      2030
           5       0.95      0.12      0.22      1118

   micro avg       0.90      0.34      0.49      9607
   macro avg       0.91      0.27      0.39      9607
weighted avg       0.90      0.34      0.47      9607
 samples avg       0.09      0.05      0.06      9607



In [27]:
rf_clf = RandomForestClassifier(n_estimators=10)

clf = MultiOutputClassifier(rf_clf)  # Replace YourClassifier with the desired classifier
clf.fit(X_train, y_train)

MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=10))

In [29]:
# Predicting Train and Test data
rf_prediction = clf.predict(X_test)
rf_pred = clf.predict(X_train)
print(" Test Accuracy {0:.2f}%".format(100*accuracy_score(rf_prediction,y_test)))
print(" Train Accuracy {0:.2f}%".format(100*accuracy_score(rf_pred, y_train)))
print(classification_report(rf_prediction,y_test))

 Test Accuracy 89.63%
 Train Accuracy 97.83%
              precision    recall  f1-score   support

           0       0.26      0.89      0.40       463
           1       0.02      0.33      0.04        12
           2       0.21      0.86      0.34       216
           3       0.03      0.33      0.05         3
           4       0.14      0.74      0.24       153
           5       0.05      0.78      0.09         9

   micro avg       0.20      0.84      0.32       856
   macro avg       0.12      0.65      0.19       856
weighted avg       0.22      0.84      0.34       856
 samples avg       0.02      0.03      0.02       856



In [30]:
xg_clf = XGBClassifier()

clf = MultiOutputClassifier(xg_clf)
clf.fit(X_train, y_train)

MultiOutputClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric=None,
                                              feature_types=None, gamma=None,
                                              grow_policy=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning_rate=None, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=None, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=None, ...))

In [31]:
# Predicting Train and Test data
xgb_prediction = clf.predict(X_test)
xgb_pred = clf.predict(X_train)
print(" Test Accuracy {0:.2f}%".format(100*accuracy_score(xgb_prediction,y_test)))
print(" Train Accuracy {0:.2f}%".format(100*accuracy_score(xgb_pred, y_train)))
print(classification_report(xgb_prediction,y_test))

 Test Accuracy 90.39%
 Train Accuracy 97.85%
              precision    recall  f1-score   support

           0       0.59      0.82      0.69      1149
           1       0.19      0.58      0.29        57
           2       0.57      0.84      0.68       603
           3       0.13      0.56      0.21         9
           4       0.48      0.72      0.58       531
           5       0.26      0.65      0.37        57

   micro avg       0.53      0.79      0.63      2406
   macro avg       0.37      0.69      0.47      2406
weighted avg       0.54      0.79      0.64      2406
 samples avg       0.05      0.06      0.05      2406

